XLNET
- https://medium.com/@harikrishnanhari.india/language-model-fine-tuning-vs-logistic-regression-for-email-spam-classification-faabcff6ff68

In [ ]:
# Reading the data from Google Drive

from google.colab import drive
import os

drive.mount('/content/drive/')

os.listdir('/content/drive/My Drive/TM Proj') # # list the contents of the directory

Mounted at /content/drive/


['XLNet.ipynb',
 'enron_clean_reduced.csv',
 'Merging Data.ipynb',
 'enron_clean_V2.csv']

In [ ]:
import pandas as pd
# Display all columns
pd.set_option('display.max_columns', None)

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/TM Proj/enron_clean_V2.csv')

# Display DataFrame df
df

,Year,Month,Day,Hour,Day_of_week,sentiment_score_compound,sentiment_score_positive,sentiment_score_neutral,sentiment_score_negative,Sender-Type,Unique-Mails-From-Sender,Contains-Reply-Forwards,Subject_lemma,Body_lemma,Subject_num_sentences,Subject_median_chars_per_word,Subject_median_words_per_sentence,Subject_uppercase_ratio,Subject_punctuation_ratio,Subject_typo_ratio,Subject_special_chars_ratio,Body_num_sentences,Body_median_chars_per_word,Body_median_words_per_sentence,Body_uppercase_ratio,Body_punctuation_ratio,Body_typo_ratio,Body_special_chars_ratio,Bcc_count,Label,Body_lexical_complexity,Body_lemma_dominant_topic,Subject_lemma_dominant_topic,VBZ,VBD,CD,PRP,VB,VBN,RB,TO,RP,VBG,NNP,VBP,JJ,NNS,IN,DT,CC,JJS,WP,NN,JJR
0,2000,11,16,17,3,0.9616,0.226,0.708,0.066,0,0.003281,0,status,status john : I be not really sure what happen...,0.0,0.063158,0.027027,0.166667,0.000000,0.000000,0.000000,0.017204,0.052632,0.012067,0.028716,0.055743,0.036036,0.055743,0.000000,0,0.923077,11,5,1,3,2,10,7,2,9,4,2,1,9,8,8,1,12,8,2,1,1,13,0
1,2000,12,8,13,4,0.5940,0.074,0.926,0.000,0,0.000729,0,re : summer inverse,re : summer inverse I suck - hope you ve make ...,0.0,0.089474,0.054054,0.000000,0.055556,0.000000,0.055556,0.004301,0.052632,0.018996,0.000000,0.030508,0.140351,0.030508,0.000000,0,0.947368,3,11,0,1,1,1,4,0,4,1,0,1,0,5,8,3,3,4,2,0,0,15,1
2,2001,5,15,16,1,-0.3923,0.036,0.918,0.045,0,0.000547,0,the wti bullet swap contract,"the wti bullet swap contract hi , follow the e...",0.0,0.042105,0.135135,0.172414,0.000000,0.200000,0.000000,0.004301,0.052632,0.062007,0.074792,0.042475,0.088889,0.042475,0.000000,0,0.775862,4,9,1,7,21,6,7,5,4,5,0,8,43,4,6,10,9,9,1,0,1,26,0
3,2000,12,12,7,1,-0.4767,0.000,0.829,0.171,0,0.001641,1,fwd : nytimes.com article : suspend rabbi quit...,fwd : nytimes.com article : suspend rabbi quit...,0.0,0.084211,0.216216,0.149254,0.044776,0.111111,0.044776,0.006452,0.111842,0.004301,0.076923,0.117949,0.310345,0.117949,0.000000,0,0.617647,2,5,0,1,1,0,1,1,0,1,0,0,10,0,2,0,1,0,0,0,0,10,0
4,2001,5,15,6,1,-0.0772,0.058,0.893,0.050,0,0.064163,0,daily chart and matrix as hot link 5/15,daily chart and matrix as hot link 5/15 the in...,0.0,0.047368,0.216216,0.000000,0.023256,0.000000,0.023256,0.017204,0.052632,0.022700,0.052726,0.035746,0.010101,0.035746,0.000000,0,0.877143,0,2,2,3,5,9,11,7,4,6,0,1,45,5,14,17,23,16,11,0,0,19,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433069,2001,9,21,18,4,0.0000,0.000,1.000,0.000,1,0.001276,0,review board book w / rebecca \n c./billb / d...,review board book w / rebecca c./billb / daveg...,0.0,0.063158,0.135135,0.229885,0.114943,0.571429,0.114943,0.002151,0.078947,0.010753,0.201058,0.126984,0.264706,0.126984,0.000000,0,0.942308,10,8,0,1,3,0,0,0,0,0,0,0,12,0,1,0,0,0,0,0,0,1,0
433070,2001,9,4,15,1,0.0000,0.000,1.000,0.000,1,0.008385,0,audit committee material meeting,audit committee material meet two meeting have...,0.0,0.084211,0.108108,0.090909,0.000000,0.000000,0.000000,0.002151,0.078947,0.024731,0.089744,0.028846,0.060000,0.028846,0.004386,0,0.818116,11,4,0,0,6,0,1,2,0,1,0,1,21,1,1,3,2,2,2,0,0,3,0
433071,2001,11,5,9,0,0.9758,0.118,0.851,0.031,1,0.011666,0,credit story,"credit story rick / bill / david , generally ,...",0.0,0.057895,0.054054,0.166667,0.000000,0.000000,0.000000,0.017204,0.052632,0.029630,0.028217,0.025465,0.011858,0.025465,0.000000,0,0.896825,11,4,6,0,0,22,16,3,15,15,2,12,14,17,18,20,23,19,10,0,0,21,3
433072,2001,10,17,23,2,0.9658,0.067,0.905,0.028,1,0.012213,0,commodity group limit issue,commodity group limit issue in addition to the...,0.0,0.052632,0.108108,0.148148,0.000000,0.000000,0.000000,0.040860,0.052632,0.029194,0.064918,0.038695,0.045139,0.038695,0.002193,0,0.902439,4,8,19,11,25,21,22,14,34,12,2,6,123,6,14,12,57,63,22,1,0,61,0


In [ ]:
# We need the below for XLNettokenizer to work

!pip  install transformers[sentencepiece]

# After installing we need to restart the kernel to see the changes

In [ ]:
from transformers import XLNetTokenizer

model_name = "textattack/xlnet-base-cased-SST-2"
tokenizer = XLNetTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

In [ ]:
# Tokenizing
train_encodings= tokenizer(train_data,  padding= True)
eval_encodings= tokenizer(eval_data,  padding= True)
test_encodings= tokenizer(test_data,  padding= True)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

# Assuming df is your DataFrame containing the email data

# Step 1: Prepare the Data
# Extract relevant features and split the data
features = ['sentiment_score_compound', 'sentiment_score_positive', 'sentiment_score_neutral', 'sentiment_score_negative',
            'Subject_lemma', 'Body_lemma', 'Label']
data = df[features]
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_texts = train_data[['Subject_lemma', 'Body_lemma']].astype(str).values.tolist()
train_labels = train_data['Label'].tolist()
test_texts = test_data[['Subject_lemma', 'Body_lemma']].values.tolist()
test_labels = test_data['Label'].tolist()


In [ ]:
# Step 2: Tokenize the Text
tokenizer = XLNetTokenizer.from_pretrained('textattack/xlnet-base-cased-SST-2')
train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Step 3: Define and Train the Model
model = XLNetForSequenceClassification.from_pretrained('textattack/xlnet-base-cased-SST-2', num_labels=2)
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir='./logs',
    output_dir='./results',
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=test_encodings,
)
trainer.train()



In [ ]:
# Step 4: Evaluate the Model
predictions = trainer.predict(test_encodings)
predicted_labels = predictions.predictions.argmax(-1)
accuracy = accuracy_score(test_labels, predicted_labels)
print("Accuracy:", accuracy)

# Step 5: Test the Model (Optional)
# If satisfied with the performance, you can use the model for inference on new data